# Perform GSEA using GSEAPY  

Following the potocol defined here: https://gseapy.readthedocs.io/en/latest/gseapy_tutorial.html#use-gsea-command-or-gsea


In [5]:
%matplotlib inline
%config InlineBackend.figure_format='retina' # mac
import pandas as pd
import gseapy as gp
import numpy as np
import matplotlib.pyplot as plt

In [6]:
gp.__version__

'0.9.16'

In [7]:
def run_GSEA_gene(celltype,nperm=1000,rseed=1000,
             rnkfile="../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.csv",
             glist='../dat/glists/gsea_final_all.gmt',col=0,
             rank_metrics='odds',**kwargs):
    '''
    col=1 - Z, 0 - logFC
    '''

    gene_exp_alpha = pd.read_csv(rnkfile)[['celltype','gene',rank_metrics]]
            
    #gene_exp_alpha.head()
    rnk = gene_exp_alpha.loc[gene_exp_alpha["celltype"]==celltype].drop(columns='celltype').sort_values(by=rank_metrics)
    if(rank_metrics=='odds'):
        rnk['odds'] =np.log2(rnk['odds'])
        rnk=rnk.reset_index(drop=True)
        df=rnk.drop(columns='gene')        
        np.random.seed(seed=rseed)
        for i in  rnk.index[(rnk['odds']==np.inf).tolist()].tolist():
            rnk.iloc[i,1]= float(df[~df.isin([np.inf])].max(0)*(1+np.random.uniform()/100))

        for i in  rnk.index[(rnk['odds']==-np.inf).tolist()].tolist():
            rnk.iloc[i,1]= float(df[~df.isin([-np.inf])].min(0)*(1+np.random.uniform()/100))
        #rnk['odds']=rnk['odds']/max(abs(rnk['odds']))
        
    print(rnk.shape)
    print(rnk.head(1))
    print(rnk.tail(1))
    
    gs_res_a = gp.prerank(rnk=rnk, # or data='./P53_resampling_data.txt'
                     gene_sets=glist, # enrichr library names or gmt file
                     #set permutation_type to phenotype if samples >=15
                     permutation_num=nperm, # reduce number to speed up test
                     outdir=None,  # do not write output to disk
                     no_plot=True, # Skip plotting
                     #weighted_score_type=1,
                     #ascending=False,
                        seed=rseed,
                     min_size=1,
                     max_size=5000,
                     processes=8,**kwargs)
                     #format='png')
    return(gs_res_a)

## Run on complete list 

In [10]:
gs_res_a=run_GSEA_gene(celltype='alpha',nperm=1000,glist='../dat/glists/gsea_final_all.gmt',weighted_score_type=0,
                      rnkfile="../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.uniq_gene.csv")
gs_res_b=run_GSEA_gene(celltype='beta',nperm=1000,glist='../dat/glists/gsea_final_all.gmt',weighted_score_type=0,
                      rnkfile="../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.uniq_gene.csv")
gs_res_d=run_GSEA_gene(celltype='delta',nperm=1000,glist='../dat/glists/gsea_final_all.gmt',weighted_score_type=0,
                      rnkfile="../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.uniq_gene.csv")
display(gs_res_a.res2d.sort_index())
display(gs_res_b.res2d.sort_index())
display(gs_res_d.res2d.sort_index())
gs_res_b.res2d.to_csv('../figures/Fig2/subfigs/GSEA_beta_res_final2.csv')
gs_res_a.res2d.to_csv('../figures/Fig2/subfigs/GSEA_alpha_res_final2.csv')
gs_res_d.res2d.to_csv('../figures/Fig2/subfigs/GSEA_delta_res_final2.csv')

(14893, 2)
      gene      odds
0  CXorf65 -5.165166
       gene     odds
14892  AVIL  3.61352
(14789, 2)
      gene      odds
0  PLA2G4E -3.910672
        gene     odds
14788  CSPG4  4.12579
(13826, 2)
    gene      odds
0  ACMSD -4.273141
       gene      odds
13825  GBP1  2.821581


2019-12-03 10:39:31,469 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.670816,40.595712,0.000000,0.000000,645,645,COL18A1;SLC51B;NHSL2;CA3;ABCA13;ALX4;NTNG2;F2;...,COL18A1;SLC51B;NHSL2;CA3;ABCA13;ALX4;NTNG2;F2;...
Alpha2,-0.744484,-94.089821,0.000000,0.000000,4508,4508,ARHGEF10L;GCK;PXN;TSHZ1;BAIAP3;LSP1;HNF4A;RHOH...,CEP85L;HIGD1A;CASP7;PLCB1;EGR1;POLR2H;CLCN2;CH...
Beta sub1_xin,0.282263,2.368440,0.218324,0.329850,13,13,FFAR4;RBP4;PPP1R1A;SCGB2A1;SCGN;PRSS23;TAGLN2;...,FFAR4;RBP4;PPP1R1A;SCGB2A1;SCGN
Beta sub2_xin,0.200641,2.453358,0.190945,0.337180,28,28,NPY;TFF3;RBP1;SCG2;GNAS;ID1;PAM;SEC11C;RPS4X;G...,NPY;TFF3;RBP1;SCG2;GNAS;ID1;PAM;SEC11C;RPS4X;G...
Beta sub3_xin,0.288027,2.121571,0.410678,0.421638,13,9,CHGA;INS;ASB9;CPE;IGFBP7;TIMP1;LAMP1;CHGB;CKB,CHGA;INS;ASB9;CPE;IGFBP7;TIMP1;LAMP1;CHGB;CKB
Beta sub4_xin,-0.141502,-6.437926,0.000000,0.000000,390,381,GCG;MMP7;HIST1H1C;AUP1;LURAP1L;TMEM176A;TMEM17...,ZNF622;UBE2D1;TAF1D;CCT8;IFRD1;PDIA4;DNAJB9;AT...
Beta1,0.381041,25.318048,0.000000,0.000000,856,837,IFI44;SLC51B;AL136376.1;PDE6C;MS4A8;LRRC14B;NW...,IFI44;SLC51B;AL136376.1;PDE6C;MS4A8;LRRC14B;NW...
Beta2,-0.246120,-15.407653,0.000000,0.000000,671,667,MLIP;PXN;TSHZ1;FAM129A;PCDH7;GPM6A;SORBS2;ANK2...,GLIS3;NRL;TPD52;DUSP4;PDHX;SMPDL3A;OMA1;SLC38A...
Delta1,0.538696,8.437340,0.000000,0.000000,39,39,PDE6C;F11;DHRS2;CDHR3;EYS;TMEM72;VWC2;SLC30A8;...,PDE6C;F11;DHRS2;CDHR3;EYS;TMEM72;VWC2;SLC30A8;...


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.367334,20.733620,0.000000,0.000000,645,638,RHBDL2;SLC43A3;CPA1;KLKB1;HSH2D;DYSF;AL136376....,RHBDL2;SLC43A3;CPA1;KLKB1;HSH2D;DYSF;AL136376....
Alpha2,-0.339816,-46.478267,0.000000,0.000000,4508,4473,DNASE1;RFX4;GPSM1;PLEKHG5;ADRA2C;HES2;MICB;AMN...,COG5;POM121;TCF4;USP54;KRBA1;ST3GAL2;LMF1;CYHR...
Beta sub1_xin,0.221867,2.094271,0.485900,0.508599,13,13,SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A;PRSS23;ASCL2;N...,SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A;PRSS23;ASCL2
Beta sub2_xin,0.272446,3.213720,0.045276,0.071345,28,27,ID1;TFF3;RBP1;ID3;IAPP;NPY;FOS;GPX3;RGS16;RPS4...,ID1;TFF3;RBP1;ID3;IAPP;NPY;FOS;GPX3;RGS16;RPS4...
Beta sub3_xin,0.222383,1.744023,0.628283,0.610262,13,10,INS;DLK1;CHGA;ASB9;LAMP1;CKB;IGFBP7;TIMP1;CPE;...,INS;DLK1;CHGA;ASB9
Beta sub4_xin,-0.180557,-7.854803,0.000000,0.000000,390,380,MMP7;MSS51;KRT18;PSMB2;AUP1;DNAJC8;SYF2;ID2;ZN...,BAX;SERP1;TMEM176A;TMEM176B;ATG12;TMEM208;PITH...
Beta1,0.620100,41.271504,0.000000,0.000000,856,856,TIGIT;ZNF541;PRSS53;DKFZP779J2370;CCR8;RHBDL2;...,TIGIT;ZNF541;PRSS53;DKFZP779J2370;CCR8;RHBDL2;...
Beta2,-0.657181,-38.848292,0.000000,0.000000,671,671,CPNE7;FOSB;DNAJB6;SORBS2;MTPAP;RAD21;PHC2;PWWP...,ZFAND5;ABCB9;USO1;G3BP2;RIMS2;UGDH;MARCH3;C1GA...
Delta1,0.425027,6.218238,0.000000,0.000000,39,39,PDE6C;GJD3;STARD8;SLC30A8;AC079354.1;CTXN3;USH...,PDE6C;GJD3;STARD8;SLC30A8;AC079354.1;CTXN3;USH...


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.327302,18.394404,0.000000,0.000000,645,608,CA3;POF1B;GBP1;CYP4F3;PPP1R1C;EDN2;LPHN3;PRMT8...,CA3;POF1B;GBP1;CYP4F3;PPP1R1C;EDN2;LPHN3;PRMT8...
Alpha2,-0.250957,-33.179831,0.000000,0.000000,4508,4410,ZFR2;DNAAF3;HAND1;CLDN11;PIEZO1;PSD2;NT5DC2;BA...,HDLBP;SEPT2;PANK1;SDK1;KIAA0922;GINM1;CDKL5;RN...
Beta sub1_xin,0.231839,1.986676,0.449304,0.499533,13,13,SCGN;FFAR4;RBP4;ASCL2;SCGB2A1;PRSS23;NPTX2;PPP...,SCGN;FFAR4;RBP4;ASCL2;SCGB2A1;PRSS23;NPTX2;PPP...
Beta sub2_xin,0.308321,3.890286,0.014644,0.012496,28,25,NENF;RBP1;GPX3;STMN2;PEMT;IGFBP5;ID1;SEC11C;FO...,NENF;RBP1;GPX3;STMN2;PEMT;IGFBP5;ID1;SEC11C;FO...
Beta sub3_xin,0.356313,2.386201,0.217899,0.376030,13,8,CHGA;ASB9;CHGB;LAMP1;IGFBP7;CKB;CPE;TIMP1,CHGA;ASB9;CHGB;LAMP1;IGFBP7;CKB
Beta sub4_xin,-0.104179,-4.568461,0.000000,0.000647,390,376,HLA-E;SST;MMP7;LRRC59;CHCHD3;PAPOLA;C19orf53;S...,TMEM176A;TMEM176B;PSMD8;GNAI1;CCNL1;STRAP;CLK1...
Beta1,0.292216,18.780862,0.000000,0.000000,856,802,STARD8;ZIC1;ANKRD29;GJD3;MOB2;FRMPD2;LPHN3;ITG...,STARD8;ZIC1;ANKRD29;GJD3;MOB2;FRMPD2;LPHN3;ITG...
Beta2,-0.199937,-12.268287,0.000000,0.000000,671,662,GPM6A;TRIM29;AHCYL1;TSHZ1;GNPTG;TSR3;TMEM156;H...,DCAF6;MPC2;KIF21A;VEGFA;ALCAM;PROX1;ACSL3;IPP;...
Delta1,0.707528,10.000731,0.000000,0.000000,39,39,STARD8;GJD3;PKD2L2;VWC2;CTXN3;F11;TNR;GLIS2;AC...,STARD8;GJD3;PKD2L2;VWC2;CTXN3;F11;TNR;GLIS2;AC...


In [28]:
gs_res_a=run_GSEA_gene(celltype='alpha',nperm=1000,glist='../dat/glists/gsea_final_all.gmt',weighted_score_type=0,
                      rnkfile="../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.uniq_gene.csv")
gs_res_b=run_GSEA_gene(celltype='beta',nperm=1000,glist='../dat/glists/gsea_final_all.gmt',weighted_score_type=0,
                      rnkfile="../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.uniq_gene.csv")
gs_res_d=run_GSEA_gene(celltype='delta',nperm=1000,glist='../dat/glists/gsea_final_all.gmt',weighted_score_type=0,
                      rnkfile="../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.uniq_gene.csv")
display(gs_res_a.res2d.sort_index())
display(gs_res_b.res2d.sort_index())
display(gs_res_d.res2d.sort_index())
gs_res_b.res2d.to_csv('../figures/Fig2/subfigs/GSEA_beta_res_final.csv')
gs_res_a.res2d.to_csv('../figures/Fig2/subfigs/GSEA_alpha_res_final.csv')
gs_res_d.res2d.to_csv('../figures/Fig2/subfigs/GSEA_delta_res_final.csv')

(14893, 2)
      gene      odds
0  CXorf65 -5.165166
       gene     odds
14892  AVIL  3.61352
(14789, 2)
      gene      odds
0  PLA2G4E -3.910672
        gene     odds
14788  CSPG4  4.12579
(13826, 2)
    gene      odds
0  ACMSD -4.273141
       gene      odds
13825  GBP1  2.821581


2019-11-26 09:22:11,647 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.670816,19.445346,0.000000,0.000000,645,645,COL18A1;SLC51B;NHSL2;CA3;ABCA13;ALX4;NTNG2;F2;...,COL18A1;SLC51B;NHSL2;CA3;ABCA13;ALX4;NTNG2;F2;...
Alpha2,-0.744484,-47.797629,0.000000,0.000000,4508,4508,ARHGEF10L;GCK;PXN;TSHZ1;BAIAP3;LSP1;HNF4A;RHOH...,CEP85L;HIGD1A;CASP7;PLCB1;EGR1;POLR2H;CLCN2;CH...
Beta sub1_xin,0.282263,1.215010,0.218324,0.295490,13,13,FFAR4;RBP4;PPP1R1A;SCGB2A1;SCGN;PRSS23;TAGLN2;...,FFAR4;RBP4;PPP1R1A;SCGB2A1;SCGN
Beta sub2_xin,0.200641,1.246306,0.190945,0.317480,28,28,NPY;TFF3;RBP1;SCG2;GNAS;ID1;PAM;SEC11C;RPS4X;G...,NPY;TFF3;RBP1;SCG2;GNAS;ID1;PAM;SEC11C;RPS4X;G...
Beta sub3_xin,0.288027,1.033205,0.410678,0.452234,13,9,CHGA;INS;ASB9;CPE;IGFBP7;TIMP1;LAMP1;CHGB;CKB,CHGA;INS;ASB9;CPE;IGFBP7;TIMP1;LAMP1;CHGB;CKB
Beta sub4_xin,-0.141502,-3.218963,0.000000,0.000000,390,381,GCG;MMP7;HIST1H1C;AUP1;LURAP1L;TMEM176A;TMEM17...,ZNF622;UBE2D1;TAF1D;CCT8;IFRD1;PDIA4;DNAJB9;AT...
Beta1,0.381041,12.583070,0.000000,0.000000,856,837,IFI44;SLC51B;AL136376.1;PDE6C;MS4A8;LRRC14B;NW...,IFI44;SLC51B;AL136376.1;PDE6C;MS4A8;LRRC14B;NW...
Beta2,-0.246120,-7.287820,0.000000,0.000000,671,667,MLIP;PXN;TSHZ1;FAM129A;PCDH7;GPM6A;SORBS2;ANK2...,GLIS3;NRL;TPD52;DUSP4;PDHX;SMPDL3A;OMA1;SLC38A...
Delta1,0.538696,3.990862,0.000000,0.000000,39,39,PDE6C;F11;DHRS2;CDHR3;EYS;TMEM72;VWC2;SLC30A8;...,PDE6C;F11;DHRS2;CDHR3;EYS;TMEM72;VWC2;SLC30A8;...


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.367334,10.574146,0.000000,0.000000,645,638,RHBDL2;SLC43A3;CPA1;KLKB1;HSH2D;DYSF;AL136376....,RHBDL2;SLC43A3;CPA1;KLKB1;HSH2D;DYSF;AL136376....
Alpha2,-0.339816,-22.495481,0.000000,0.000000,4508,4473,DNASE1;RFX4;GPSM1;PLEKHG5;ADRA2C;HES2;MICB;AMN...,COG5;POM121;TCF4;USP54;KRBA1;ST3GAL2;LMF1;CYHR...
Beta sub1_xin,0.221867,0.965459,0.485900,0.552727,13,13,SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A;PRSS23;ASCL2;N...,SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A;PRSS23;ASCL2
Beta sub2_xin,0.272446,1.632570,0.045276,0.063341,28,27,ID1;TFF3;RBP1;ID3;IAPP;NPY;FOS;GPX3;RGS16;RPS4...,ID1;TFF3;RBP1;ID3;IAPP;NPY;FOS;GPX3;RGS16;RPS4...
Beta sub3_xin,0.222383,0.863292,0.628283,0.625268,13,10,INS;DLK1;CHGA;ASB9;LAMP1;CKB;IGFBP7;TIMP1;CPE;...,INS;DLK1;CHGA;ASB9
Beta sub4_xin,-0.180557,-4.037369,0.000000,0.000000,390,380,MMP7;MSS51;KRT18;PSMB2;AUP1;DNAJC8;SYF2;ID2;ZN...,BAX;SERP1;TMEM176A;TMEM176B;ATG12;TMEM208;PITH...
Beta1,0.620100,20.429394,0.000000,0.000000,856,856,TIGIT;ZNF541;PRSS53;DKFZP779J2370;CCR8;RHBDL2;...,TIGIT;ZNF541;PRSS53;DKFZP779J2370;CCR8;RHBDL2;...
Beta2,-0.657181,-19.307601,0.000000,0.000000,671,671,CPNE7;FOSB;DNAJB6;SORBS2;MTPAP;RAD21;PHC2;PWWP...,ZFAND5;ABCB9;USO1;G3BP2;RIMS2;UGDH;MARCH3;C1GA...
Delta1,0.425027,3.189956,0.000000,0.000000,39,39,PDE6C;GJD3;STARD8;SLC30A8;AC079354.1;CTXN3;USH...,PDE6C;GJD3;STARD8;SLC30A8;AC079354.1;CTXN3;USH...


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.327302,9.086836,0.000000,0.000000,645,608,CA3;POF1B;GBP1;CYP4F3;PPP1R1C;EDN2;LPHN3;PRMT8...,CA3;POF1B;GBP1;CYP4F3;PPP1R1C;EDN2;LPHN3;PRMT8...
Alpha2,-0.250957,-15.992678,0.000000,0.000000,4508,4410,ZFR2;DNAAF3;HAND1;CLDN11;PIEZO1;PSD2;NT5DC2;BA...,HDLBP;SEPT2;PANK1;SDK1;KIAA0922;GINM1;CDKL5;RN...
Beta sub1_xin,0.231839,0.999298,0.449304,0.498560,13,13,SCGN;FFAR4;RBP4;ASCL2;SCGB2A1;PRSS23;NPTX2;PPP...,SCGN;FFAR4;RBP4;ASCL2;SCGB2A1;PRSS23;NPTX2;PPP...
Beta sub2_xin,0.308321,1.859557,0.014644,0.024709,28,25,NENF;RBP1;GPX3;STMN2;PEMT;IGFBP5;ID1;SEC11C;FO...,NENF;RBP1;GPX3;STMN2;PEMT;IGFBP5;ID1;SEC11C;FO...
Beta sub3_xin,0.356313,1.226508,0.217899,0.340812,13,8,CHGA;ASB9;CHGB;LAMP1;IGFBP7;CKB;CPE;TIMP1,CHGA;ASB9;CHGB;LAMP1;IGFBP7;CKB
Beta sub4_xin,-0.104179,-2.348189,0.000000,0.000216,390,376,HLA-E;SST;MMP7;LRRC59;CHCHD3;PAPOLA;C19orf53;S...,TMEM176A;TMEM176B;PSMD8;GNAI1;CCNL1;STRAP;CLK1...
Beta1,0.292216,9.521897,0.000000,0.000000,856,802,STARD8;ZIC1;ANKRD29;GJD3;MOB2;FRMPD2;LPHN3;ITG...,STARD8;ZIC1;ANKRD29;GJD3;MOB2;FRMPD2;LPHN3;ITG...
Beta2,-0.199937,-5.790631,0.000000,0.000000,671,662,GPM6A;TRIM29;AHCYL1;TSHZ1;GNPTG;TSR3;TMEM156;H...,DCAF6;MPC2;KIF21A;VEGFA;ALCAM;PROX1;ACSL3;IPP;...
Delta1,0.707528,5.320389,0.000000,0.000000,39,39,STARD8;GJD3;PKD2L2;VWC2;CTXN3;F11;TNR;GLIS2;AC...,STARD8;GJD3;PKD2L2;VWC2;CTXN3;F11;TNR;GLIS2;AC...


## tests

In [24]:
for score_type in [0,1,1.4,2]:
    print(score_type)
    gs_res_b=run_GSEA_gene(celltype='beta',nperm=1000,glist='../dat/glists/gsea_final_all.gmt',
                           weighted_score_type=score_type,
                           rnkfile="../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.uniq_gene.csv")
    fdr=gs_res_b.res2d.loc['mutli_anticorrelated'].fdr
    print('fdr for scoretype {0}:{1}'.format(score_type,fdr))


0
(14789, 2)
      gene      odds
0  PLA2G4E -3.910672
        gene     odds
14788  CSPG4  4.12579
fdr for scoretype 0:0.025846065971726402
1
(14789, 2)
      gene      odds
0  PLA2G4E -3.910672
        gene     odds
14788  CSPG4  4.12579
fdr for scoretype 1:0.23377960865087538
1.4
(14789, 2)
      gene      odds
0  PLA2G4E -3.910672
        gene     odds
14788  CSPG4  4.12579
fdr for scoretype 1.4:0.40843507214206437
2
(14789, 2)
      gene      odds
0  PLA2G4E -3.910672
        gene     odds
14788  CSPG4  4.12579
fdr for scoretype 2:0.6075027995520716


In [22]:
for i in range(10):
    print(i)
    gs_res_b=run_GSEA_gene(celltype='beta',nperm=1000,glist='../dat/glists/gsea_final_all.gmt',rseed=i
                      rnkfile="../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.uniq_gene.csv")
    fdr=gs_res_b.res2d.loc['mutli_anticorrelated'].fdr
    print(fdr)
    if(fdr<.1):
        break


0
(14789, 2)
      gene      odds
0  PLA2G4E -3.927591
        gene      odds
14788  CSPG4  4.107433
0.24259166248116523
1
(14789, 2)
      gene      odds
0  PLA2G4E -3.905989
        gene      odds
14788  CSPG4  4.096108
0.26751269035532993
2
(14789, 2)
      gene      odds
0  PLA2G4E -3.915276
        gene      odds
14788  CSPG4  4.107299
0.2725925925925926
3
(14789, 2)
      gene      odds
0  PLA2G4E -3.937362
        gene      odds
14788  CSPG4  4.126628
0.2605445808862787
4
(14789, 2)
      gene      odds
0  PLA2G4E -3.910818
        gene      odds
14788  CSPG4  4.118643
0.2297979797979798
5
(14789, 2)
      gene      odds
0  PLA2G4E -3.926258
        gene      odds
14788  CSPG4  4.110082
0.2542372881355932
6
(14789, 2)
      gene      odds
0  PLA2G4E -3.925607
        gene      odds
14788  CSPG4  4.094509
0.22227899846704138
7
(14789, 2)
      gene    odds
0  PLA2G4E -3.9234
        gene      odds
14788  CSPG4  4.130106
0.24339720352149147
8
(14789, 2)
      gene     odds
0  PLA2

In [29]:
%%bash 
jupyter nbconvert --execute  --to notebook --inplace plot_GSEA.ipynb

[NbConvertApp] WARNING | Config option `template_path` not recognized by `NotebookExporter`.
[NbConvertApp] Converting notebook plot_GSEA.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: ir
[NbConvertApp] Writing 55665 bytes to plot_GSEA.ipynb


## save data 

In [11]:
from gseapy.plot import gseaplot, heatmap
terms = gs_res_b.res2d.index
terms

Index(['Alpha1', 'Alpha2', 'Beta sub4_xin', 'Beta1', 'Beta2', 'Delta1',
       'Delta2', 'Mawla_Beta_2', 'mutli_anticorrelated', 'Mawla_Beta_1',
       'Beta sub2_xin', 'multi_correlated', 'Beta sub1_xin', 'Beta sub3_xin'],
      dtype='object', name='Term')

In [13]:
from gseapy.plot import gseaplot, heatmap
terms = gs_res_b.res2d.index
for tm in terms:
    gseaplot(gs_res_b.ranking, term=tm, 
             **gs_res_b.results[tm],ofname=tm+'_beta_all.pdf')

terms = gs_res_a.res2d.index
for tm in terms:
    gseaplot(gs_res_a.ranking, term=tm, 
             **gs_res_a.results[tm],ofname=tm+'_alpha_all.pdf')

terms = gs_res_d.res2d.index
for tm in terms:
    gseaplot(gs_res_d.ranking, term=tm, 
             **gs_res_d.results[tm],ofname=tm+'_delta_all.pdf')
